# Inference with the PyTorch implementation

**Note: `aenet-python` needs to be installed with the `[torch]` requirements (`pip install aenet[torch]`) for this notebook to work.**

Predicting energies and forces with the PyTorch API is nearly identical to the Fortran/Python API.

In [1]:
from aenet.torch_training.trainer import TorchANNPotential

pot = TorchANNPotential.from_file('pt-TiO2/Ti-O.pt')
# 'cpu' is the default devide. Send to GPU by explicitly specifying
# pot = TorchANNPotential.from_file('pt-TiO2/Ti-O.pt', device='cuda')

results = pot.predict(
    ['xsf-TiO2/structure-001.xsf'],
    eval_forces=True
)

results

PredictOut with 1 structure(s)  
  Structure 0: 23 atoms, E_tot = -19516.791 eV = -848.556 eV/atom  


Various predicted properties, including the total and cohesive energy, can be accessed via the `PredictOut` class.  Computation of the forces is optional and has to be requested.

In [2]:
results?

Type:        PredictOut
String form:
PredictOut with 1 structure(s)
  Structure 0: 23 atoms, E_tot = -19516.791 eV = -848.556 eV/atom
File:        ~/Documents/Cline/aenet-python/src/aenet/io/predict.py
Docstring:  
Parser and representation of output files generated by 'predict.x'.

Attributes
----------
coords : List[np.ndarray]
    Cartesian coordinates for each structure.
forces : List[np.ndarray]
    Atomic forces for each structure (if computed).
atom_types : List[np.ndarray]
    Atomic species symbols for each structure.
atom_energies : List[np.ndarray]
    Atomic energies for each structure (if computed).
cohesive_energy : List[float]
    Cohesive energies in eV/atom for each structure.
total_energy : List[float]
    Total energies in eV for each structure.
inputs : PredictIn or None
    Associated input configuration if predict_in_path was provided.

The atom energies are also only returned when they are requested.  As for the Fortran API, this and other options can be set using `PredictionConfig` objects.

In [3]:
from aenet.mlip import PredictionConfig
from aenet.torch_training.trainer import TorchANNPotential

pot = TorchANNPotential.from_file('pt-TiO2/Ti-O.pt')

results = pot.predict(
    ['xsf-TiO2/structure-001.xsf'],
    eval_forces=True,
    config=PredictionConfig(
        print_atomic_energies=True,
        timing=True
    )
)

print(results.atom_energies)

[array([-0.71563512, -0.71347665, -0.74569544, -0.74239586, -0.73947   ,
       -0.77258344, -0.69833605, -0.72072413, -0.15845261, -0.17792292,
        0.06419335, -0.13566747, -0.14608   , -0.1394532 , -0.16032179,
       -0.16988787, -0.14831708, -0.14311656, -0.16197844, -0.16550643,
       -0.16633298, -0.17048792, -0.16349779])]


We also requested timing information (all values in seconds):

In [4]:
results.timing

{'featurization': [0.0],
 'energy_eval': [0.00020122528076171875],
 'force_eval': [0.0046880245208740234],
 'total': [0.004979133605957031]}

## Processing large numbers of structures

For larger number of structures, `TorchANNPotential` implements batch-processing and parallelization with PyTorch workers.  Per default, the batch size is set to 32, but it can be adjusted via a config option.

- `num_workers` (int): Number of DataLoader workers. Default: 0
- `prefetch_factor` (int): Number of batches to prefetch per worker. Default: 2
- `persistent_workers` (bool): Whether to keep DataLoader workers alive between epochs. Default: True

**Note: Don't expect these parameters to have a huge impact on CPUs.**

In [5]:
import glob
import numpy as np

from aenet.mlip import PredictionConfig
from aenet.torch_training.trainer import TorchANNPotential

# this time we load all 100 structures
xsf_files = glob.glob('./xsf-TiO2/*.xsf')
pot = TorchANNPotential.from_file('pt-TiO2/Ti-O.pt')

In [6]:
%%time

# parameters optimized for a 2021 M1 MacBook Pro
# with 4 performance and 4 efficiency cores
cfg = PredictionConfig(
    batch_size=4,
    num_workers=3,
    prefetch_factor=2,
    persistent_workers=True,
)

results = pot.predict(
    xsf_files,
    eval_forces=True,
    config=cfg
)

CPU times: user 494 ms, sys: 189 ms, total: 684 ms
Wall time: 8.97 s


Let's compare this with the default options.

In [7]:
%%time

results = pot.predict(
    xsf_files,
    eval_forces=True,
)

CPU times: user 13.9 s, sys: 1.95 s, total: 15.8 s
Wall time: 15.8 s


On our reference laptop, the spead-up is about a factor of two for this small data set.

## Comparison with inference from aenet-Fortran

On CPUs, the Fortran-based compiled aenet tools are significantly more efficient.

**Note: the following cell requires aenet's Fortran tool's to be correctly installed and configured for use with Python**

In [8]:
import glob
import numpy as np

from aenet.mlip import PredictionConfig, ANNPotential
from aenet.torch_training.trainer import TorchANNPotential

# load PyTorch model and save it in the 
# Fortran-compatible ASCII format
torch_pot = TorchANNPotential.from_file('pt-TiO2/Ti-O.pt')
torch_pot.to_aenet_ascii('./pt-TiO2/')

# load all 100 structures again
xsf_files = glob.glob('./xsf-TiO2/*.xsf')
fort_pot = ANNPotential.from_files(
    {'Ti': './pt-TiO2/potential.Ti.nn.ascii',
     'O': './pt-TiO2/potential.O.nn.ascii'},
    potential_format='ascii')

In [9]:
%%time

torch_results = torch_pot.predict(
    xsf_files,
    eval_forces=True,
)

CPU times: user 13.9 s, sys: 2 s, total: 15.9 s
Wall time: 15.7 s


In [10]:
%%time

fort_results = fort_pot.predict(
    xsf_files,
    eval_forces=True,
)

CPU times: user 8.41 ms, sys: 13.8 ms, total: 22.2 ms
Wall time: 788 ms


On our test machine (2021 MacBook Pro), Fortran is around 20 times faster than PyTorch for this small dataset.

The results are, of course, (numerically) identical.

In [11]:
import numpy as np
diff = np.array(torch_results.total_energy) - np.array(fort_results.total_energy)
print("Difference: {:.3e} ± {:.3e} eV/structure".format(diff.mean(), diff.std()))

Difference: 1.679e-10 ± 2.988e-09 eV/structure
